In [1]:
import re
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
##전역 변수
vocab_size = 4000           #단어의 종류
sentence_len = 70           #한 문장의 최대 단어 숫자
tag_size = 10               #태그의 종류
vocab_len = 52               #한 단어의 최대 문자 숫자
char_size = 93              #문자의 종류

## 파일을 읽고 

In [3]:
def readfile(filename):
    f = open(filename, 'r')
    tagged_sentences = []
    sentence = []

    for line in f:
        if len(line)==0 or line.startswith('-DOCSTART') or line[0]=="\n":
            if len(sentence) > 0:
                tagged_sentences.append(sentence)
                sentence = []
            continue
        splits = line.split(' ') # 공백을 기준으로 속성을 구분한다.
        splits[-1] = re.sub(r'\n', '', splits[-1]) # 줄바꿈 표시 \n을 제거한다.
        #word = splits[0].lower() # 단어들은 소문자로 바꿔서 저장한다.
        sentence.append([splits[0], splits[-1]]) # 단어와 개체명 태깅만 기록한다.
    return tagged_sentences

In [4]:
trainSentences=readfile("train.txt")

In [5]:
validSentences=readfile("valid.txt")
testSentences=readfile("test.txt")

## 문장과 태그 값을 쪼개어 각각 다른 array 저장

In [6]:
def seperatearray(rawsentence):
    sentences, ner_tags = [], [] 
    for tagged_sentence in rawsentence: # 14,041개의 문장 샘플을 1개씩 불러온다.
        sentence, tag_info = zip(*tagged_sentence) # 각 샘플에서 단어들은 sentence에 개체명 태깅 정보들은 tag_info에 저장.
        sentences.append(list(sentence)) # 각 샘플에서 단어 정보만 저장한다.
        ner_tags.append(list(tag_info)) # 각 샘플에서 개체명 태깅 정보만 저장한다.
    return sentences, ner_tags

In [7]:
train_sentence, train_tag = seperatearray(trainSentences)
valid_sentence, valid_tag = seperatearray(validSentences)
test_sentence, test_tag = seperatearray(testSentences)

In [8]:
train_sentence[:3]

[['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'],
 ['Peter', 'Blackburn'],
 ['BRUSSELS', '1996-08-22']]

## 문장은 문장대로(sentence_len) 단어는 단어대로(vocab_len) padding

In [9]:
def sentencetochar(sentences):
    charpaddedsentence=[]
    for sentence in sentences:
        newSentence=[]
        makesentence =[]
        makesentence.extend(sentence)
        while len(makesentence)<sentence_len:        #문장 padding
            makesentence.append("#")
        if len(makesentence)>sentence_len:           #문장 MAX 길이에 따라 자름
            makesentence=makesentence[:sentence_len]
        for words in makesentence:              
            if len(words) > vocab_len:          #단어 padding
                words=words[:vocab_len]
            newSentence.append([words.ljust(vocab_len,'#')])    #단어 MAX길이에 따라 자름
            

        charpaddedsentence.append(newSentence)
    return charpaddedsentence

In [10]:
train_char= sentencetochar(train_sentence)

valid_char = sentencetochar(valid_sentence)
test_char = sentencetochar(test_sentence)

## char2Idx 딕셔너리 생성

In [11]:
char_to_index={"#":0,"UNKNOWN":1}
for c in "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,-_()[]{}!?:;'\"/\\%$`&=*+@^~|":
    char_to_index[c]=len(char_to_index)

In [12]:
char_size = len(char_to_index)

## char단위로 쪼개며 char2Idx딕셔너리 대로 int 값으로 변경

In [13]:
def addCharInformation(Sentences):
    for i,sentence in enumerate(Sentences):
        for j,data in enumerate(sentence):
            chars=[c for c in data[0]]            # Character 분리
            Sentences[i][j]=chars 
    return Sentences

In [14]:
train_char = addCharInformation(train_char)
valid_char = addCharInformation(valid_char)
test_char = addCharInformation(test_char)

In [15]:
def addCharInformation2(Sentences):
    total=[]
    for i,sentence in enumerate(Sentences):
        Sen=[]
        for j,data in enumerate(sentence):
            changeInt=[]
            for k, chars in enumerate(data):
                changeInt.append(char_to_index[chars])
            Sen.append(changeInt) # 단어, Chracter, NER을 리스트로
        total.append(Sen)
    return total

In [16]:
X_char_train = addCharInformation2(train_char)
X_char_valid = addCharInformation2(valid_char)
X_char_test = addCharInformation2(test_char)

## np.array 형변환

In [17]:
X_char_train = np.array([np.array(x1) for x1 in X_char_train])
X_char_valid = np.array([np.array(x1) for x1 in X_char_valid])
X_char_test = np.array([np.array(x1) for x1 in X_char_test])

In [18]:
X_char_train.shape

(14041, 70, 52)

## 단어 및 태그 처리

In [19]:
max_words = 4000        #문장 데이터에 있는 모든 단어를 사용하지 않고 높은 빈도수를 가진 상위 약 4,000개의 단어만을 사용
src_tokenizer = Tokenizer(num_words=max_words, oov_token='OOV')         #tokenizer 객체 생성
src_tokenizer.fit_on_texts(train_sentence)                              #인덱스 구축 

tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(train_tag)

In [20]:
tar_tokenizer

In [21]:
vocab_size = max_words
tag_size = len(tar_tokenizer.word_index) + 1
print('단어 집합의 크기 : {}'.format(vocab_size))
print('개체명 태깅 정보 집합의 크기 : {}'.format(tag_size))

단어 집합의 크기 : 4000
개체명 태깅 정보 집합의 크기 : 10


In [22]:
X_train = src_tokenizer.texts_to_sequences(train_sentence)
y_train = tar_tokenizer.texts_to_sequences(train_tag)

In [23]:
print(train_sentence[:3])

[['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], ['Peter', 'Blackburn'], ['BRUSSELS', '1996-08-22']]


In [24]:
X_valid = src_tokenizer.texts_to_sequences(valid_sentence)
y_valid = tar_tokenizer.texts_to_sequences(valid_tag)

X_test = src_tokenizer.texts_to_sequences(test_sentence)
y_test = tar_tokenizer.texts_to_sequences(test_tag)

In [25]:
X_train[:3]

[[989, 1, 205, 629, 7, 3939, 216, 1, 3], [774, 1872], [726, 150]]

In [26]:
index_to_word = src_tokenizer.index_word
index_to_ner = tar_tokenizer.index_word


In [27]:
index_to_ner

{1: 'o',
 2: 'b-loc',
 3: 'b-per',
 4: 'b-org',
 5: 'i-per',
 6: 'i-org',
 7: 'b-misc',
 8: 'i-loc',
 9: 'i-misc'}

In [28]:
decoded = []
for index in X_test[0] : # 첫번째 샘플 안의 인덱스들에 대해서
    decoded.append(index_to_word[index]) # 다시 단어로 변환

print('기존 문장 : {}'.format(test_sentence[0]))
print('빈도수가 낮은 단어가 OOV 처리된 문장 : {}'.format(decoded))

기존 문장 : ['SOCCER', '-', 'JAPAN', 'GET', 'LUCKY', 'WIN', ',', 'CHINA', 'IN', 'SURPRISE', 'DEFEAT', '.']
빈도수가 낮은 단어가 OOV 처리된 문장 : ['soccer', '-', 'japan', 'get', 'OOV', 'win', ',', 'china', 'in', 'surprise', 'defeat', '.']


In [29]:
X_train = pad_sequences(X_train, padding='post', maxlen=sentence_len)
# X_train의 모든 샘플들의 길이를 맞출 때 뒤의 공간에 숫자 0으로 채움.
y_train = pad_sequences(y_train, padding='post', maxlen=sentence_len)
# y_train의 모든 샘플들의 길이를 맞출 때 뒤의 공간에 숫자0으로 채움.
X_valid = pad_sequences(X_valid, padding='post', maxlen=sentence_len)
y_valid = pad_sequences(y_valid, padding='post', maxlen=sentence_len)
X_test = pad_sequences(X_test, padding='post', maxlen=sentence_len)
y_test = pad_sequences(y_test, padding='post', maxlen=sentence_len)

In [30]:
y_train = to_categorical(y_train, num_classes=tag_size)
y_valid = to_categorical(y_valid, num_classes=tag_size)
y_test = to_categorical(y_test, num_classes=tag_size)           ##원핫 인코딩 시킴 

In [31]:
X_train.shape

(14041, 70)

# ==================<< 모델 생성 >>================
# 모델 1. Character CNN 추가한 functional API
- CNN 1D

In [34]:
words_input = tf.keras.layers.Input(shape=(None, ),dtype='int32', name='modelInput')
words = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128, mask_zero=True)(words_input)

In [35]:
character_input=tf.keras.layers.Input(shape=(None,vocab_len,),name='char_input')
character_input

<tf.Tensor 'char_input:0' shape=(None, None, 52) dtype=float32>

In [34]:
character_input=tf.keras.layers.Input(shape=(None,vocab_len,),name='char_input')
embed_char_out=tf.keras.layers.TimeDistributed(tf.keras.layers.Embedding(len(char_to_index),30,embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= tf.keras.layers.Dropout(0.5)(embed_char_out)
conv1d_out= tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))(dropout)
maxpool_out=tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling1D(vocab_len))(conv1d_out)
char = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(maxpool_out)
char = tf.keras.layers.Dropout(0.5)(char)

In [35]:
char.shape

TensorShape([None, None, 30])

In [36]:
output = tf.keras.layers.concatenate([words, char])
output = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True), merge_mode='concat')(output)
output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(tag_size, activation='softmax'))(output)

In [37]:
model = tf.keras.Model(inputs=[words_input, character_input], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='nadam',metrics=[tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         [(None, None, 52)]   0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 2790        char_input[0][0]                 
__________________________________________________________________________________________________
dropout (Dropout)               (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, None, 52, 30) 2730        dropout[0][0]                    
______________________________________________________________________________________________

In [38]:
model.fit(x=[X_train, X_char_train],y=y_train,epochs=8,batch_size=128,validation_data=([X_valid, X_char_valid], y_valid))

Train on 14041 samples, validate on 3250 samples
Epoch 1/8
14041/14041 [==============================] - 21s 1ms/sample - loss: 0.1258 - precision: 0.8971 - recall: 0.7929 - val_loss: 0.0888 - val_precision: 0.9379 - val_recall: 0.8319
Epoch 2/8
14041/14041 [==============================] - 10s 683us/sample - loss: 0.0542 - precision: 0.9585 - recall: 0.8882 - val_loss: 0.0524 - val_precision: 0.9614 - val_recall: 0.9065
Epoch 3/8
14041/14041 [==============================] - 10s 679us/sample - loss: 0.0328 - precision: 0.9693 - recall: 0.9397 - val_loss: 0.0343 - val_precision: 0.9730 - val_recall: 0.9447
Epoch 4/8
14041/14041 [==============================] - 10s 680us/sample - loss: 0.0251 - precision: 0.9738 - recall: 0.9557 - val_loss: 0.0325 - val_precision: 0.9754 - val_recall: 0.9472
Epoch 5/8
14041/14041 [==============================] - 10s 682us/sample - loss: 0.0210 - precision: 0.9770 - recall: 0.9627 - val_loss: 0.0310 - val_precision: 0.9707 - val_recall: 0.9521
Epo

In [39]:
loss,pre,recall = model.evaluate([X_test,X_char_test], y_test,verbose=0)
f1_val = 2*(pre*recall)/(pre+recall)
print("\n 테스트 f-1 score: %.4f" % f1_val)
print("\n 테스트 정확도 : %.4f" % (1-loss))

# print(model.evaluate([X_test,X_char_test], y_test,verbose=0))


 테스트 f-1 score: 0.9541

 테스트 정확도 : 0.9696


# =======================================================================
# 모델 2. Subclassing Model 
- CNN 1D 
- Subclassing

In [40]:
class TestModel (tf.keras.Model):
    def __init__(self,vocab_size,tag_size):
        super().__init__()
        self.WordEmbedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128, mask_zero=True)
        self.CharEmbedding = tf.keras.layers.TimeDistributed(tf.keras.layers.Embedding(len(char_to_index),30,embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')
        self.CharConv1D = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))
        self.CharDropout1 = tf.keras.layers.Dropout(0.5)
        self.CharMaxpool = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling1D(52))
        self.CharFlatten = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())
        self.CharDropout2 = tf.keras.layers.Dropout(0.5)
        self.fBiLSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True), merge_mode='concat')
        self.fTimeDistributed = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(tag_size, activation='softmax'))

    def call(self, inp):
        wrd = self.WordEmbedding(inp[0])
        chrc = self.CharEmbedding(inp[1])
        chrc = self.CharDropout1(chrc)
        chrc = self.CharConv1D(chrc)
        chrc = self.CharMaxpool(chrc)
        chrc = self.CharFlatten(chrc)
        chrc = self.CharDropout2(chrc)
        x = tf.keras.layers.concatenate([wrd,chrc])
        # x= tf.concat([wrd,chrc],axis=-1)
        x = self.fBiLSTM(x)
        x = self.fTimeDistributed(x)
        return x


In [41]:
modelSub = TestModel(vocab_size,tag_size)
# words_input = tf.keras.layers.Input(shape=(None, ),dtype='int32', name='wordInput')
# character_input=tf.keras.layers.Input(shape=(None,52,),name='char_input')             ###위에 선언 되어있음! 
# modelSub.summary()

In [42]:
modelSub([words_input,character_input])

<tf.Tensor 'test_model/Identity:0' shape=(None, None, 10) dtype=float32>

In [43]:
# modelSub(words_input,character_input)
modelSub.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=[tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')])
modelSub.summary()

Model: "test_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         512000    
_________________________________________________________________
char_embedding (TimeDistribu (None, None, 52, 30)      2790      
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 52, 30)      2730      
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 52, 30)      0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, None, 1, 30)       0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, None, 30)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 30)          0

In [44]:
modelSub.fit(x=[X_train, X_char_train],y=y_train,epochs=8,batch_size=128,validation_data=([X_valid, X_char_valid], y_valid))

Train on 14041 samples, validate on 3250 samples
Epoch 1/8
14041/14041 [==============================] - 18s 1ms/sample - loss: 0.1282 - precision: 0.8952 - recall: 0.7900 - val_loss: 0.0891 - val_precision: 0.9492 - val_recall: 0.8250
Epoch 2/8
14041/14041 [==============================] - 10s 682us/sample - loss: 0.0554 - precision: 0.9581 - recall: 0.8870 - val_loss: 0.0539 - val_precision: 0.9560 - val_recall: 0.9050
Epoch 3/8
14041/14041 [==============================] - 10s 682us/sample - loss: 0.0315 - precision: 0.9701 - recall: 0.9424 - val_loss: 0.0326 - val_precision: 0.9752 - val_recall: 0.9468
Epoch 4/8
14041/14041 [==============================] - 10s 689us/sample - loss: 0.0231 - precision: 0.9766 - recall: 0.9590 - val_loss: 0.0312 - val_precision: 0.9750 - val_recall: 0.9490
Epoch 5/8
14041/14041 [==============================] - 10s 678us/sample - loss: 0.0196 - precision: 0.9792 - recall: 0.9655 - val_loss: 0.0268 - val_precision: 0.9782 - val_recall: 0.9559
Epo

In [45]:
loss,pre,recall = modelSub.evaluate([X_test,X_char_test], y_test,verbose=0)
f1_val = 2*(pre*recall)/(pre+recall)
print("\n 테스트 f-1 score: %.4f" % f1_val)
print("\n 테스트 정확도 : %.4f" % (1-loss))

print(modelSub.evaluate([X_test,X_char_test], y_test,verbose=0))


 테스트 f-1 score: 0.9567

 테스트 정확도 : 0.9705
[0.029523488811127974, 0.9672619, 0.94637734]


# ========================================================================================
# 모델 3. CNN을 추가한 모델
- Conv 2D

In [46]:
class TestModel2D (tf.keras.Model):
    def __init__(self,vocab_size,tag_size):
        super().__init__()
        self.WordEmbedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128, mask_zero=True)
        self.CharEmbedding = tf.keras.layers.TimeDistributed(tf.keras.layers.Embedding(len(char_to_index),30,embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')
        self.CharEmbedding2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Embedding(len(char_to_index),30,embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')
        self.CharConv2D = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(kernel_size=3, filters=6, padding='same',activation='tanh', strides=1))
        self.CharDropout1 = tf.keras.layers.Dropout(0.5)
        self.CharMaxpool = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D([52,30]))
        self.CharFlatten = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())
        self.CharDropout2 = tf.keras.layers.Dropout(0.5)
        self.fBiLSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True), merge_mode='concat')
        self.fTimeDistributed = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(tag_size, activation='softmax'))

    def call(self, inp):
        wrd = self.WordEmbedding(inp[0])
        chrc = self.CharEmbedding(inp[1])
        chrc = self.CharEmbedding2(chrc)
        chrc = self.CharDropout1(chrc)
        chrc = self.CharConv2D(chrc)
        chrc = self.CharMaxpool(chrc)
        chrc = self.CharFlatten(chrc)
        chrc = self.CharDropout2(chrc)
        # x = tf.keras.layers.concatenate([wrd,chrc])
        x= tf.concat([wrd,chrc],axis=-1)
        x = self.fBiLSTM(x)
        x = self.fTimeDistributed(x)
        return x


In [47]:
modelSub2D = TestModel2D(vocab_size,tag_size)
modelSub2D([words_input,character_input])
# modelSub.summary()

<tf.Tensor 'test_model2d/Identity:0' shape=(None, None, 10) dtype=float32>

In [48]:
modelSub2D.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=[tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')])
modelSub2D.summary()

Model: "test_model2d"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 128)         512000    
_________________________________________________________________
char_embedding (TimeDistribu (None, None, 52, 30)      2790      
_________________________________________________________________
char_embedding (TimeDistribu (None, None, 52, 30, 30)  2790      
_________________________________________________________________
time_distributed_8 (TimeDist (None, None, 52, 30, 6)   1626      
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 52, 30, 30)  0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, None, 1, 1, 6)     0         
_________________________________________________________________
time_distributed_10 (TimeDis (None, None, 6)          

In [49]:
modelSub2D.fit(x=[X_train, X_char_train],y=y_train,epochs=8,batch_size=128,validation_data=([X_valid, X_char_valid], y_valid))

Train on 14041 samples, validate on 3250 samples
Epoch 1/8
14041/14041 [==============================] - 169s 12ms/sample - loss: 0.2563 - precision: 0.9541 - recall: 0.9168 - val_loss: 0.1459 - val_precision: 0.9667 - val_recall: 0.9619
Epoch 2/8
14041/14041 [==============================] - 164s 12ms/sample - loss: 0.1010 - precision: 0.9878 - recall: 0.9623 - val_loss: 0.0999 - val_precision: 0.9934 - val_recall: 0.9539
Epoch 3/8
14041/14041 [==============================] - 165s 12ms/sample - loss: 0.0747 - precision: 0.9920 - recall: 0.9673 - val_loss: 0.0785 - val_precision: 0.9878 - val_recall: 0.9693
Epoch 4/8
14041/14041 [==============================] - 165s 12ms/sample - loss: 0.0569 - precision: 0.9918 - recall: 0.9754 - val_loss: 0.0657 - val_precision: 0.9879 - val_recall: 0.9761
Epoch 5/8
14041/14041 [==============================] - 167s 12ms/sample - loss: 0.0430 - precision: 0.9923 - recall: 0.9830 - val_loss: 0.0538 - val_precision: 0.9905 - val_recall: 0.9788
E

In [50]:
loss,pre,recall = modelSub2D.evaluate([X_test,X_char_test], y_test,verbose=0)
f1_val = 2*(pre*recall)/(pre+recall)
print("\n 테스트 f-1 score: %.4f" % f1_val)
print("\n 테스트 정확도 : %.4f" % (1-loss))

print(modelSub2D.evaluate([X_test,X_char_test], y_test,verbose=0))


 테스트 f-1 score: 0.9867

 테스트 정확도 : 0.9537
[0.046318683964776154, 0.98919743, 0.9842373]


# ========================================
# 모델 4.
## TimeDistributed layer 빼기
(feat. 간략화)

In [51]:
class TestModel3 (tf.keras.Model):
    def __init__(self,vocab_size,tag_size):
        super().__init__()
        self.WordEmbedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128, mask_zero=True)
        self.CharDropout1 = tf.keras.layers.Dropout(0.5)
        self.CharConv1D =tf.keras.layers.Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, input_shape=(70,52))
        self.fBiLSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True), merge_mode='concat')
        self.fTimeDistributed = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(tag_size, activation='softmax'))

    def call(self, inp):
        wrd = self.WordEmbedding(inp[0])
        chrc = self.CharDropout1(inp[1])
        chrc = self.CharConv1D(chrc)
        x = tf.keras.layers.concatenate([wrd,chrc])
        x = self.fBiLSTM(x)
        x = self.fTimeDistributed(x)
        return x

In [52]:
model3 = TestModel3(vocab_size,tag_size)
model3([words_input,character_input])
# modelSub.summary()

<tf.Tensor 'test_model3/Identity:0' shape=(None, None, 10) dtype=float32>

In [53]:
model3.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=[tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')])
model3.summary()

Model: "test_model3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 128)         512000    
_________________________________________________________________
dropout_6 (Dropout)          (None, None, 52)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 30)          4710      
_________________________________________________________________
bidirectional_3 (Bidirection (None, None, 512)         849920    
_________________________________________________________________
time_distributed_12 (TimeDis (None, None, 10)          5130      
Total params: 1,371,760
Trainable params: 1,371,760
Non-trainable params: 0
_________________________________________________________________


In [54]:
model3.fit(x=[X_train, X_char_train],y=y_train,epochs=8,batch_size=128,validation_data=([X_valid, X_char_valid], y_valid))

Train on 14041 samples, validate on 3250 samples
Epoch 1/8
14041/14041 [==============================] - 12s 890us/sample - loss: 0.1301 - precision: 0.9005 - recall: 0.7762 - val_loss: 0.0784 - val_precision: 0.9496 - val_recall: 0.8545
Epoch 2/8
14041/14041 [==============================] - 4s 264us/sample - loss: 0.0564 - precision: 0.9593 - recall: 0.8835 - val_loss: 0.0522 - val_precision: 0.9528 - val_recall: 0.9176
Epoch 3/8
14041/14041 [==============================] - 4s 260us/sample - loss: 0.0364 - precision: 0.9649 - recall: 0.9332 - val_loss: 0.0434 - val_precision: 0.9578 - val_recall: 0.9331
Epoch 4/8
14041/14041 [==============================] - 4s 260us/sample - loss: 0.0288 - precision: 0.9690 - recall: 0.9494 - val_loss: 0.0382 - val_precision: 0.9628 - val_recall: 0.9421
Epoch 5/8
14041/14041 [==============================] - 4s 257us/sample - loss: 0.0243 - precision: 0.9728 - recall: 0.9580 - val_loss: 0.0377 - val_precision: 0.9651 - val_recall: 0.9387
Epoch

In [55]:
loss,pre,recall = model3.evaluate([X_test,X_char_test], y_test,verbose=0)
f1_val = 2*(pre*recall)/(pre+recall)
print("\n 테스트 f-1 score: %.4f" % f1_val)
print("\n 테스트 정확도 : %.4f" % (1-loss))

# print(model3.evaluate([X_test,X_char_test], y_test,verbose=0))


 테스트 f-1 score: 0.9390

 테스트 정확도 : 0.9542


# 모델 5. Conv2D
### Embedding 사용하여 차원 맞추기

In [74]:
class TestModel5 (tf.keras.Model):
    def __init__(self,vocab_size,tag_size):
        super().__init__()
        self.WordEmbedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128, mask_zero=True)
        self.CharEmbedding = tf.keras.layers.Embedding(len(char_to_index),30,embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5), mask_zero=True)
        self.CharDropout1 = tf.keras.layers.Dropout(0.5)
        self.CharConv2D =tf.keras.layers.Conv2D(kernel_size=(3,52), filters=30, padding='same',activation='tanh', strides=1, input_shape=(70,52,30))
        # self.CharMaxpooling =  tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool1D(52))
        self.CharMaxpooling =  tf.keras.layers.MaxPooling2D(pool_size=(1,52))
        #self.CharFlatten = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())
        # self.CharFlatten = tf.keras.layers.Flatten(data_format='channels_last')
        self.fBiLSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True), merge_mode='concat')
        self.fTimeDistributed = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(tag_size, activation='softmax'))

    def call(self, inp):
        wrd = self.WordEmbedding(inp[0])
        chrc = self.CharEmbedding(inp[1])
        chrc = self.CharDropout1(chrc)
        chrc = self.CharConv2D(chrc)
        chrc = self.CharMaxpooling(chrc)
        # chrc = self.CharFlatten(chrc)
        chrc = tf.squeeze(chrc)
        x = tf.keras.layers.concatenate([wrd,chrc])
        x = self.fBiLSTM(x)
        x = self.fTimeDistributed(x)
        return x

In [75]:
model5 = TestModel5(vocab_size,tag_size)
model5([words_input,character_input])
# modelSub.summary()

ValueError: in converted code:

    <ipython-input-74-c62db5486241>:23 call  *
        x = tf.keras.layers.concatenate([wrd,chrc])
    C:\Users\ezcare14\Anaconda3\envs\shyun\lib\site-packages\tensorflow_core\python\keras\layers\merge.py:705 concatenate
        return Concatenate(axis=axis, **kwargs)(inputs)
    C:\Users\ezcare14\Anaconda3\envs\shyun\lib\site-packages\tensorflow_core\python\keras\engine\base_layer.py:817 __call__
        self._maybe_build(inputs)
    C:\Users\ezcare14\Anaconda3\envs\shyun\lib\site-packages\tensorflow_core\python\keras\engine\base_layer.py:2141 _maybe_build
        self.build(input_shapes)
    C:\Users\ezcare14\Anaconda3\envs\shyun\lib\site-packages\tensorflow_core\python\keras\utils\tf_utils.py:305 wrapper
        input_shape = convert_shapes(input_shape, to_tuples=True)
    C:\Users\ezcare14\Anaconda3\envs\shyun\lib\site-packages\tensorflow_core\python\keras\utils\tf_utils.py:231 convert_shapes
        input_shape)
    C:\Users\ezcare14\Anaconda3\envs\

In [70]:
model5.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=[tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')])
model5.summary()

Model: "test_model5_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, None, 128)         512000    
_________________________________________________________________
embedding_22 (Embedding)     (None, None, 52, 30)      2790      
_________________________________________________________________
dropout_10 (Dropout)         (None, None, 52, 30)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, None, 52, 30)      140430    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, None, 1, 30)       0         
_________________________________________________________________
time_distributed_20 (TimeDis (None, None, 30)          0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, None, 512)      

In [71]:
model5.fit(x=[X_train, X_char_train],y=y_train,epochs=8,batch_size=128,validation_data=([X_valid, X_char_valid], y_valid))

Train on 14041 samples, validate on 3250 samples
Epoch 1/8
14041/14041 [==============================] - 28s 2ms/sample - loss: 0.1221 - precision: 0.8980 - recall: 0.8021 - val_loss: 0.0787 - val_precision: 0.9448 - val_recall: 0.8503
Epoch 2/8
14041/14041 [==============================] - 17s 1ms/sample - loss: 0.0443 - precision: 0.9725 - recall: 0.9030 - val_loss: 0.0359 - val_precision: 0.9779 - val_recall: 0.9348
Epoch 3/8
14041/14041 [==============================] - 17s 1ms/sample - loss: 0.0224 - precision: 0.9826 - recall: 0.9560 - val_loss: 0.0257 - val_precision: 0.9816 - val_recall: 0.9567
Epoch 4/8
14041/14041 [==============================] - 17s 1ms/sample - loss: 0.0159 - precision: 0.9856 - recall: 0.9704 - val_loss: 0.0222 - val_precision: 0.9816 - val_recall: 0.9631
Epoch 5/8
14041/14041 [==============================] - 17s 1ms/sample - loss: 0.0131 - precision: 0.9872 - recall: 0.9761 - val_loss: 0.0212 - val_precision: 0.9805 - val_recall: 0.9669
Epoch 6/8
1

In [60]:
loss,pre,recall = model5.evaluate([X_test,X_char_test], y_test,verbose=0)
f1_val = 2*(pre*recall)/(pre+recall)
print("\n 테스트 f-1 score: %.4f" % f1_val)
print("\n 테스트 정확도 : %.4f" % (1-loss))



 테스트 f-1 score: 0.9600

 테스트 정확도 : 0.9729


# =================================================================
# 모델 6. SeparableConv1D


In [77]:
class TestModel6 (tf.keras.Model):
    def __init__(self,vocab_size,tag_size):
        super().__init__()
        self.WordEmbedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128, mask_zero=True)
        self.CharDropout1 = tf.keras.layers.Dropout(0.5)
        self.CharConv1D =tf.keras.layers.SeparableConv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, input_shape=(70,52))
        self.fBiLSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True), merge_mode='concat')
        self.fTimeDistributed = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(tag_size, activation='softmax'))

    def call(self, inp):
        wrd = self.WordEmbedding(inp[0])
        chrc = self.CharDropout1(inp[1])
        chrc = self.CharConv1D(chrc)
        x = tf.keras.layers.concatenate([wrd,chrc])
        x = self.fBiLSTM(x)
        x = self.fTimeDistributed(x)
        return x

In [78]:
model6 = TestModel6(vocab_size,tag_size)
model6([words_input,character_input])
# modelSub.summary()

<tf.Tensor 'test_model6/Identity:0' shape=(None, None, 10) dtype=float32>

In [79]:
model6.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=[tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')])
model6.summary()

Model: "test_model6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, None, 128)         512000    
_________________________________________________________________
dropout_13 (Dropout)         (None, None, 52)          0         
_________________________________________________________________
separable_conv1d (SeparableC (None, None, 30)          1746      
_________________________________________________________________
bidirectional_13 (Bidirectio (None, None, 512)         849920    
_________________________________________________________________
time_distributed_24 (TimeDis (None, None, 10)          5130      
Total params: 1,368,796
Trainable params: 1,368,796
Non-trainable params: 0
_________________________________________________________________


In [80]:
model6.fit(x=[X_train, X_char_train],y=y_train,epochs=8,batch_size=128,validation_data=([X_valid, X_char_valid], y_valid))

Train on 14041 samples, validate on 3250 samples
Epoch 1/8
  128/14041 [..............................] - ETA: 4:42

AssertionError: 